In [1]:
import numpy as np
import cv2
import win32api
from pymouse import PyMouse
from imutils.video import WebcamVideoStream
import win32api, win32con
import dlib
from imutils import face_utils
from scipy.spatial import distance as dist

In [2]:
#Threshold for Eye Aspect Ratio(Detecting a closed eye)
earThreshold=0.15

# Threshold for a valid eyeBrow raise
BrowThreshold=5
    
#Threshold for blink duration(For ajusting left click senstivity)
clickThresholdL=3

#Threshold for eyebrow raises(For adjusting right click senstivity)
clickThresholdR=4
    
# initialize accumulated weight
accumWeight = 0.5

#mouse Speed
speed=20
    
#move threshold
moveThreshold=3

In [3]:
def eyeBrow_threshold(eyeB1,eyeB2,eye1,eye2):
    avg=0
    for i in range(4):
        avg+=dist.euclidean(eyeB1[i],eye1[i])
    for i in range(4):
        avg+= dist.euclidean(eyeB2[i],eye2[i])
        
    avg/=8
    
    return float(avg)

In [4]:
def right_click(x,y):
    win32api.SetCursorPos((x,y))
    win32api.mouse_event(win32con.MOUSEEVENTF_RIGHTDOWN,x,y,0,0)
    win32api.mouse_event(win32con.MOUSEEVENTF_RIGHTUP,x,y,0,0)

In [5]:
def left_click(x,y):
    win32api.SetCursorPos((x,y))
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTDOWN,x,y,0,0)
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTUP,x,y,0,0)

In [6]:
def eye_aspect_ratio(eye):
    # compute the euclidean distances between the two sets of
    # vertical eye landmarks (x, y)-coordinates
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
 
    # compute the euclidean distance between the horizontal
    # eye landmark (x, y)-coordinates
    C = dist.euclidean(eye[0], eye[3])
 
    # compute the eye aspect ratio
    ear = (A + B) / (2.0 * C)
 
    # return the eye aspect ratio
    return ear

In [7]:
# For wink detection(pre trained dlib model)
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("./data/shape_predictor_68_face_landmarks.dat")  # use link to get file https://drive.google.com/open?id=1r84y_rYPyGuGiuJhXTjtPQHCpCj6w_Ct

# grab the indexes of the facial landmarks for the left and
# right eye, respectively
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

#indxes of facial landmarks for left and right eyebrows
(lBStart, lBEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eyebrow"]
(rBStart, rBEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eyebrow"]

In [9]:
eye_cascade1 = cv2.CascadeClassifier('xml/haarcascade_mcs_eyepair_big.xml')
eye_cascade2 = cv2.CascadeClassifier('xml/haarcascade_mcs_eyepair_small.xml')
eye_cascade3 = cv2.CascadeClassifier('xml/haarcascade_eye_tree_eyeglasses.xml')
eye_cascade4 = cv2.CascadeClassifier('xml/haarcascade_eye.xml')

window_size = 5
windowX = []
windowY = []

cap = WebcamVideoStream(src=0).start()

#Initial eye2eyeBrow distance
initialR=0

#initial Eye Aspect Ratio
initialL=0
#Initial eye2eyeBrow and aspect ratio calibration frames counter
initCounter=0

#count for closed eyes in frames to detect a valid blink
bCount=0

#count for closed eyes in frames to detect a valid eyeBrow Raise
rCount=0

mx = 0
my = 0

upper_mx = 0
lower_mx = 0

upper_my = 0
lower_my = 0

start_flag = True

while True:
    img = cap.read()
    
    m=PyMouse()
    
    #print img
    
    img = cv2.flip(img, 1)
    clone = img

    cv2.normalize(img, img, 0, 255, cv2.NORM_MINMAX)

    img_yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)

    # equalize the histogram of the Y channel
    img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])

    # convert the YUV image back to RGB format
    img = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    gray = clahe.apply(gray)
    
    eyes = eye_cascade1.detectMultiScale(gray)
    if len(eyes) == 0:
        eyes = eye_cascade2.detectMultiScale(gray)

    area = 0
    x = 0
    y = 0
    w = 0
    h = 0
        
    for (ex,ey,ew,eh) in eyes:
        if ew*eh > area:
            x = ex
            y = ey
            h = eh
            w = ew
            area = w*h

    if area > 0:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
        cv2.rectangle(img,(x+w/2-5,y+h/2-5),(x+w/2+5,y+h/2+5),(255,0,0),2)
        mx = x + w/2
        my = y + h/2
        roi_gray = gray[y-20:y+h+20, x-20:x+w+20]
        roi_color = img[y-20:y+h+20, x-20:x+w+20]

        
        if start_flag == True:
            upper_mx = mx + 35
            lower_mx = mx - 20
            upper_my = my + 35
            lower_my = my - 20
            start_flag = False
            
        mx = np.interp(mx,[lower_mx,upper_mx],[0,1366])
        my = np.interp(my,[lower_my,upper_my],[0,768])

        windowX.append(mx)
        windowY.append(my)

        if len(windowX) > window_size:
            del windowX[0]
        if len(windowY) > window_size:
            del windowY[0]

        mx = int(np.mean(windowX))
        my = int(np.mean(windowY))
        
        win32api.SetCursorPos((mx,my))
        
        rects = detector(clone, 0)
            
            # loop over the face detections
        for rect in rects:
        # determine the facial landmarks for the face region, then
        # convert the facial landmark (x, y)-coordinates to a NumPy
        # array
            shape = predictor(clone, rect)
            shape = face_utils.shape_to_np(shape)

            # extract the left and right eye coordinates, then use the
            # coordinates to compute the eye aspect ratio for both eyes
            leftEye = shape[lStart:lEnd]
            rightEye = shape[rStart:rEnd]
            leftEAR = eye_aspect_ratio(leftEye)
            rightEAR = eye_aspect_ratio(rightEye)

            # compute the convex hull for the left and right eye, then
            # visualize each of the eyes
            leftEyeHull = cv2.convexHull(leftEye)
            rightEyeHull = cv2.convexHull(rightEye)
            cv2.drawContours(clone, [leftEyeHull], -1, (0, 255, 0), 1)
            cv2.drawContours(clone, [rightEyeHull], -1, (0, 255, 0), 1)

            if (leftEAR+rightEAR)/2>=initialL and initCounter>30:
                bCount=0
            else:
                bCount+=1

            if bCount>clickThresholdL and initCounter>30:          #left click if bCount is greater then click Threshold
                m.click(m.position()[0],m.position()[1],1)
                bCount=0

            # extract the left and right eyebrow coordinates, then use the
            # coordinates to compute the eyebrow distance from eyes
            leftEyeBrow = shape[lBStart:lBEnd]
            rightEyeBrow = shape[rBStart:rBEnd]

            # compute the convex hull for the left and right eyebrow, then
            # visualize each of the eyesbrows
            leftEyeBrowHull = cv2.convexHull(leftEyeBrow)
            rightEyeBrowHull = cv2.convexHull(rightEyeBrow)
            cv2.drawContours(clone, [leftEyeBrowHull], -1, (0, 255, 0), 1)
            cv2.drawContours(clone, [rightEyeBrowHull], -1, (0, 255, 0), 1)

            eye2eyeBrowDist= eyeBrow_threshold(leftEyeBrow,rightEyeBrow,leftEye,rightEye)

            if initCounter<30:                   # For calibrating initial eye2eyeBrow distance
                initialR+=eye2eyeBrowDist
                initialL+= (leftEAR+rightEAR)/2
                initCounter+=1
            elif initCounter==30:
                initialR/=30
                initialL/=30
                initialL*=0.7             # used to set initial E.A.R threshold i.e 0.7 of initial
                initCounter+=1

            else:
                if eye2eyeBrowDist<BrowThreshold+ initialR:
                    rCount=0
                else:
                    rCount+=1

                if rCount>clickThresholdR:          #right click if rCount is greater then click Threshold
                    m.click(m.position()[0],m.position()[1],2)
                    rCount=0

            if initCounter<=30:
                cv2.putText(clone, "Left click Threshold=Calibrating", (10, 75), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)
            else:
                cv2.putText(clone, "Left click Threshold="+str(initialL), (10, 75), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)

            cv2.putText(clone, "Current Left="+str((leftEAR+rightEAR)/2), (10, 105), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)

            if initCounter<=30:
                cv2.putText(clone, "Right click Threshold=Calibrating", (10, 135), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)
            else:
                cv2.putText(clone, "Right click Threshold="+str(initialR+ BrowThreshold), (10, 135), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)

            cv2.putText(clone, "Current Right=" +str(eye2eyeBrowDist), (10, 165), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)

            cv2.putText(clone, "Quit= q" , (600, 465), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)
            cv2.putText(clone, "Reset= r" , (600, 495), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)



    img = cv2.resize(img, (0,0), fx=0.7, fy=0.7)
    cv2.imshow('img',img)
    #print "mx: " + str(mx) + " ! my: " + str(my)
    print initCounter
    if cv2.waitKey(1) == ord('q'):
        break
    
cap.stop()
cv2.destroyAllWindows()

error: OpenCV(3.4.1) C:\projects\opencv-python\opencv\modules\imgproc\src\color.cpp:11180: error: (-215) scn == 3 || scn == 4 in function cv::cvtColor
